# Word Counter

In [1]:
!pip install pandas
!pip install konlpy
!pip install sklearn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. 라이브러리 import 및 데이터 불러오기

In [3]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# 시 데이터 불러오기
poem = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nlp3/nlp_poem.csv", index_col=False)
poem.head()


,author,title,content
0,문정희,가을 노트,<br><br><br>그대 떠나간 후<br>나의 가을은<br>조금만 건드려도<br>...
1,이순,가을비 오는 날은,<br>자정이 되어 간신히 그치는 비<br>간신히 버리는 그리움<br>그동안 너무 ...
2,이성선,가을 편지,<br>잎이 떨어지고 있습니다<br>원고지처럼 하늘이 한 칸씩<br>비어 가고 있습...
3,박제영,가을에는,<br>가을에는 잠시 여행을 떠날 일이다<br>그리 수선스러운 준비는 하지 말고<b...
4,최영미,가을에는,<br>내가 그를 사랑한 것도 아닌데<br>미칠 듯 그리워질 때가 있다<br>바람의...


## 2. 데이터 전처리

In [5]:
# 불용어
stopwords = [
    '하다', '이다', '아니다', '않다', '되다', '있다', '없다', '싶다', '알다', '보다', '마다', '같다', '지다', '가다', '오다', '두다',
    '나', '내', '날', '난', '제', '너', '네', '널', '넌', '그대', '당신', '우리', '이', '그', '저', '무엇', '이런', '저런', '그런', '것', '게', '건', 
    '하나', '모든', '때문', '다음', '위해', '사람', '줄', '못', '그리다', '되어다', '수', '사랑', '말', '안', '속', '때', '머', '알', '또', '번', '왜', '뿐'
]

In [6]:
# 토큰화 함수
def tokenize(text):
    okt = Okt()

    # 줄바꿈(<br>) 제거
    text = text.replace("<br>", " ")

    # 형태소(품사) 분석 - 정규화有, 어간추출有
    okt_pos = okt.pos(text, norm=True, stem=True)

    # 특정 품사(명사, 동사, 형용사)만 사용
    lst_word = [word for word, pos in okt_pos if pos in ["Noun", "Adjective", "Verb"]]

    # 불용어 제거
    lst_word = [word for word in lst_word if word not in stopwords]

    # list를 이어 string으로 반환
    str_word = " ".join(lst_word)

    return str_word


In [7]:
# 시 내용에 토큰화 적용
content = poem["content"]
content_tokens = content.apply(tokenize)
content_tokens.head()

0    떠나가다 후 가을 조금 건드리다 우수수 몸 떨다 다한 다한 노래 끄다 씨앗 가슴 담...
1    자정 그치다 비 버리다 그리움 그동안 많다 버리다 허다 세월 지나 돌아보다 버리다 ...
2    잎 떨어지다 원고지 하늘 칸 비어 빈 곳 맑은 영혼 잉크 물 편지 써다 보내다 함 ...
3    가을 잠시 여행 떠나다 일이 수선스럽다 준비 가깝다 멀다 데 가을 스스로 높다 푸른...
4    미치다 듯 그립다 바람 손 가지런하다 풀다 뭉게구름 떼 구름 새털구름 아무 모양 만...
Name: content, dtype: object

# 3. Word Counter

In [8]:
# 하나의 문자열로 합치기
content_joined = ' '.join(content_tokens).split()
content_joined

['떠나가다',
 '후',
 '가을',
 '조금',
 '건드리다',
 '우수수',
 '몸',
 '떨다',
 '다한',
 '다한',
 '노래',
 '끄다',
 '씨앗',
 '가슴',
 '담다',
 '지고',
 '잖다',
 '잎',
 '잎',
 '아픔',
 '사라지다',
 '기억',
 '남아',
 '벼',
 '베다',
 '빈',
 '들다',
 '녘',
 '넉',
 '볏단',
 '놓이다',
 '조용하다',
 '물이',
 '들다',
 '아무',
 '홀로',
 '찬',
 '바람',
 '흔들리다',
 '세상',
 '끝나다',
 '가장',
 '깊다',
 '살속',
 '담다',
 '떠나가다',
 '후',
 '가을',
 '조금',
 '건드리다',
 '우수수',
 '옷',
 '벗다',
 '슬프다',
 '앙상하다',
 '뼈',
 '남다',
 '자정',
 '그치다',
 '비',
 '버리다',
 '그리움',
 '그동안',
 '많다',
 '버리다',
 '허다',
 '세월',
 '지나',
 '돌아보다',
 '버리다',
 '그립다',
 '짐',
 '버리다',
 '나르다',
 '버리다',
 '것임',
 '마침내',
 '깨닫다',
 '가을비',
 '꽃',
 '떨다',
 '꽃',
 '뜨다',
 '곁',
 '나다',
 '떨다',
 '서다',
 '자정',
 '지나',
 '다시',
 '내리다',
 '비',
 '다시',
 '불다',
 '들이다',
 '그리움',
 '이제',
 '허전하다',
 '지상',
 '보이지',
 '별',
 '더듬다',
 '가을비',
 '창가',
 '서서',
 '잠',
 '깨다',
 '채',
 '서다',
 '마음',
 '빗물',
 '고이',
 '듯',
 '고요하다',
 '불다',
 '잎',
 '떨어지다',
 '원고지',
 '하늘',
 '칸',
 '비어',
 '빈',
 '곳',
 '맑은',
 '영혼',
 '잉크',
 '물',
 '편지',
 '써다',
 '보내다',
 '함',
 '오히려',
 '아무렇다',
 '돌리다',
 '보내다',
 '어제',
 '다시',
 '이르다',
 '해도',

In [9]:
from collections import Counter

# 각 단어를 카운트하기
content_cnt = Counter(content_joined)
content_cnt

Counter({'떠나가다': 38,
         '후': 65,
         '가을': 149,
         '조금': 78,
         '건드리다': 19,
         '우수수': 10,
         '몸': 475,
         '떨다': 53,
         '다한': 10,
         '노래': 187,
         '끄다': 83,
         '씨앗': 28,
         '가슴': 355,
         '담다': 66,
         '지고': 61,
         '잖다': 5,
         '잎': 132,
         '아픔': 69,
         '사라지다': 252,
         '기억': 236,
         '남아': 80,
         '벼': 2,
         '베다': 33,
         '빈': 130,
         '들다': 597,
         '녘': 33,
         '넉': 7,
         '볏단': 1,
         '놓이다': 35,
         '조용하다': 75,
         '물이': 59,
         '아무': 167,
         '홀로': 66,
         '찬': 34,
         '바람': 612,
         '흔들리다': 132,
         '세상': 466,
         '끝나다': 65,
         '가장': 185,
         '깊다': 217,
         '살속': 2,
         '옷': 66,
         '벗다': 102,
         '슬프다': 168,
         '앙상하다': 11,
         '뼈': 66,
         '남다': 150,
         '자정': 11,
         '그치다': 24,
         '비': 242,
         '버리다': 453,
         

In [10]:
content_cnt.most_common()

[('눈', 652),
 ('꽃', 625),
 ('바람', 612),
 ('들다', 597),
 ('마음', 587),
 ('소리', 542),
 ('생각', 492),
 ('몸', 475),
 ('나르다', 475),
 ('길', 472),
 ('더', 468),
 ('세상', 466),
 ('곳', 463),
 ('버리다', 453),
 ('손', 448),
 ('그렇다', 440),
 ('밤', 425),
 ('위', 423),
 ('하늘', 410),
 ('모르다', 400),
 ('내다', 399),
 ('시간', 397),
 ('일', 384),
 ('나무', 382),
 ('얼굴', 377),
 ('좋다', 371),
 ('가슴', 355),
 ('다시', 347),
 ('집', 344),
 ('물', 326),
 ('살다', 313),
 ('살', 312),
 ('아름답다', 305),
 ('흐르다', 304),
 ('나다', 297),
 ('별', 290),
 ('죽다', 288),
 ('새', 286),
 ('멀다', 283),
 ('눈물', 283),
 ('어떻다', 275),
 ('앉다', 273),
 ('먹다', 268),
 ('나오다', 264),
 ('사이', 262),
 ('이름', 258),
 ('만나다', 254),
 ('듯', 253),
 ('오늘', 253),
 ('사라지다', 252),
 ('거리', 252),
 ('어둠', 250),
 ('뒤', 249),
 ('두', 247),
 ('서다', 246),
 ('어디', 244),
 ('지금', 243),
 ('비', 242),
 ('떨어지다', 242),
 ('떠나다', 239),
 ('앞', 239),
 ('자다', 239),
 ('끝', 239),
 ('기억', 236),
 ('뜨다', 234),
 ('구름', 226),
 ('바다', 224),
 ('내리다', 223),
 ('걸다', 223),
 ('놓다', 223),
 ('붉다', 221),
 ('개', 220)

In [11]:
df = pd.DataFrame(content_cnt.most_common())
df

,0,1
0,눈,652
1,꽃,625
2,바람,612
3,들다,597
4,마음,587
...,...,...
15252,라이트,1
15253,흑인,1
15254,읊조리다,1
15255,피스톤,1


In [12]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/nlp3/most_common.csv ', index=False, encoding = 'utf-8')